In [1]:
!pip install pyspark

In [2]:
import shutil, time
from pyspark.sql import SparkSession
session= SparkSession.builder.appName("Streaming_DeptBudget").getOrCreate()

In [3]:
dept_data=session.read.json("Departmentdata.json")

In [4]:
print("Schema of dataset:\n",dept_data.schema)
print("\nInformation of columns:",dept_data.columns)
print("\nNumber of records:",dept_data.count())

Schema of dataset:
 StructType([StructField('budget', LongType(), True), StructField('deptname', StringType(), True), StructField('employees', ArrayType(StructType([StructField('employee_id', LongType(), True), StructField('name', StructType([StructField('fname', StringType(), True), StructField('lname', StringType(), True)]), True), StructField('position', StringType(), True), StructField('salary', LongType(), True)]), True), True), StructField('manager', StringType(), True)])

Information of columns: ['budget', 'deptname', 'employees', 'manager']

Number of records: 12


In [5]:
# Create a folder named DeptDir
! mkdir DeptDir

In [6]:
stream_json=session.readStream.schema(dept_data.schema).json("DeptDir/")

In [7]:
# Copy the json file to the DeptDir folder
src=r"Departmentdata.json"
dest = r"DeptDir"
shutil.copy(src,dest)

'DeptDir/Departmentdata.json'

In [8]:
stream_json.writeStream.queryName("DeptTable").format("memory").outputMode("append").start()

In [9]:
# Pause 5 secconds before entering the simulation for ensuring the Spark read data
print("Waiting for Spark to process the initial file...")
time.sleep(5)

# Simulate data continues to enter DeptDir/
print("Start Spark streaming simulation")
for i in range(10):
    print("Iteration:",i+1)
    session.sql("SELECT deptname, SUM(budget) AS Total_Budget FROM DeptTable GROUP BY deptname").show()
    newfile="DeptDir/Dept" + str(i) +  ".json"
    shutil.copy(src,newfile)
    time.sleep(6)

Waiting for Spark to process the initial file...
Start Spark streaming simulation
Iteration: 1
+----------------+------------+
|        deptname|Total_Budget|
+----------------+------------+
|           Sales|          12|
|     Engineering|           9|
|  HumanResources|           8|
|         Finance|           6|
|        Research|          13|
|       Marketing|           7|
|      Production|          14|
|           Legal|          11|
|              IT|          15|
|QualityAssurance|           9|
| CustomerService|          14|
|      Operations|           7|
+----------------+------------+

Iteration: 2
+----------------+------------+
|        deptname|Total_Budget|
+----------------+------------+
|           Sales|          24|
|     Engineering|          18|
|  HumanResources|          16|
|         Finance|          12|
|      Production|          28|
|        Research|          26|
|           Legal|          22|
|       Marketing|          14|
|              IT|         